In [ ]:
import ast
import csv
import psycopg2
import src.DataClasses as dc

In [ ]:
non_json_columns = ['id', 'budget', 'homepage', 'imdb_id', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date',
                    'runtime', 'status', 'tagline', 'title', 'revenue']
ind_columns = ['genres', 'belongs_to_collection', 'production_companies', 'keywords']
dep_columns = ['cast', 'crew']
no_id_columns = ['spoken_languages', 'production_countries']
json_columns = ind_columns + dep_columns + no_id_columns
column_dataclass_map = {'genres': dc.Genre, 'belongs_to_collection': dc.Collection, 'production_companies': dc.ProductionCompany, 
                        'production_countries': dc.Country, 'spoken_languages': dc.Language, 'keywords': dc.Keyword, 'cast': dc.Cast,
                        'crew': dc.Crew}
csv_member_map = {'id': 'id', 'belongs_to_collection': 'collection_id', 'budget': 'budget', 'genres': 'genre_ids', 
                  'homepage': 'homepage', 'imdb_id': 'imdb_id', 'original_language': 'original_language_id', 
                  'original_title': 'original_title' 'overview': 'overview', 'popularity': 'popularity', 'poster_path': 'poster_path',
                  'production_companies': 'production_company_ids', 'production_countries': 'production_country_ids', 
                  'release_date': 'release_date', 'runtime': 'runtime', 'spoken_languages': 'spoken_language_ids', 'status': 'status',
                  'tagline': 'tagline', 'title': 'title', 'keywords': 'keyword_ids', 'cast': 'cast_ids', 'crew': 'crew_ids', 
                  'revenue': 'revenue'}

In [ ]:
data = []
with open('data/train.csv', 'r') as raw_data:
    reader = csv.DictReader(raw_data, delimiter=',', quotechar='"')
    for row in reader:
        entry = {}
        for key, value in row.items():
            if key in json_columns:
                if (value == '' or value == '#N/A'):
                    entry[key] = []
                else:
                    entry[key] = ast.literal_eval(value)
            else:
                entry[key] = value
        data.append(entry)

In [ ]:
conn = psycopg2.connect("host=delliott-pg dbname=delliott user=delliott password=delliott")
for item in data:
    movie = dc.Movie(**{k: v for k, v in item.items() if k in non_json_columns})
    for column in json_columns:
        cur = conn.cursor()
        for obj in item[column]:
            insert_statement = ''
            if key in dep_columns:
                insert_statement = value(movie_id=item['id'], **obj).get_insert_statement()
            else:
                insert_statement = value(**obj).get_insert_statement()
            try:
                cur.execute(insert_statement)
            except TypeError as e:
                print(f'Error: {e}\n String: {key}:{item[key]}')
                conn.rollback()
                break
            except psycopg2.errors.UniqueViolation:
                conn.rollback()
                continue
            except Exception as e:
                print(f'Error: {e}\nObject: {key}:{obj}')
                conn.rollback()
                continue
            movie[]
        conn.commit()
conn.close()

In [ ]:
conn.commit()

In [ ]:
import psycopg2
import csv
conn = psycopg2.connect("host=delliott-pg dbname=delliott user=delliott password=delliott")
cur = conn.cursor()
with open('data/train.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')
    next(reader)
    for row in reader:
        cur.execute(
        "INSERT INTO tmdb_box_office VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row)
conn.commit()